### Wednesday, December 27, 2023

Trying this again in the hfpt_Dec14 container. Yup! It all runs!

### Wednesday, December 13, 2023

Let's play with the newest model, shall we ... this is a brand new model, just out this morning!!

[mistralai/Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)

Interesting ... the notebook fails on v0.2 as soon as we try to perform inference! .. 

### Friday, December 1, 2023

docker container start hfpt_Oct28

https://colab.research.google.com/drive/1F2PeWl5FOHv4sjd7XTEu40JjqbFhC3LB?usp=sharing

In [1]:
import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

# Run Date: Wednesday, December 27, 2023
# Run Time: 00:03:44

# Run Date: Wednesday, December 13, 2023
# Run Time: 00:03:18

In [2]:
# model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [3]:
# !ls /home/rob/Data2/huggingface/transformers
!ls /root/.cache/huggingface/hub

models--TheBloke--Llama-2-13b-Chat-GPTQ      version.txt
models--mistralai--Mistral-7B-Instruct-v0.2


In [4]:
# docker cp /home/rob/Data3/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.2 c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 14.5GB to c8324b70601d://root/.cache/huggingface/hub

In [5]:
!ls /root/.cache/huggingface/hub

models--TheBloke--Llama-2-13b-Chat-GPTQ      version.txt
models--mistralai--Mistral-7B-Instruct-v0.2


In [6]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-v0.1


In [7]:
#  !pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip install -q datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate xformers einops
# !pip -q install langchain

In [8]:
!nvidia-smi

Wed Dec 27 22:05:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        On  | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P0              N/A /  70W |    443MiB /  2048MiB |     28%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Mistral 7B Instruct



In [9]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
# Load the model if it's not in this container
# mistralai/Mistral-7B-Instruct-v0.1
# docker cp /home/rob/Data3/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.1 c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 15GB to c9b676310ea0://home/rob/Data2/huggingface/transformers


In [12]:
# Load the model if it's not in this container
# mistralai/Mistral-7B-Instruct-v0.2
# docker cp /home/rob/Data3/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.2 c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 14.5GB to c9b676310ea0://home/rob/Data2/huggingface/transformers

In [13]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [14]:
# We don't need to run this
# torch.set_default_device('cuda')

In [15]:
# model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
# model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype="auto")

In [17]:

# This does not work ... ALL VRAM is sucked up almost immediately ...
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", )

# This works! 5478 MiB of VRAM ... but then fails when we try to move the model to the device a few cells below ...
# So to use this, you need to comment out any code that attempts to move the model to the gpu .. 
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_4bit=True )

# This attempts to load the model to CPU RAM
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# This slowly sucks up CPU RAM, taking up about 16gb (4.9gb to 20.9gb), then drops it into the GPU VRAM taking up 14966 MiB
# 2m 0.2s
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

# This immediately sucks up all the VRAM!
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# This fails on the dtype parameter
# model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.bfloat16)

# Download and load time for ...
# mistralai/Mistral-7B-Instruct-v0.2
# CPU times: user 57.2 s, sys: 1min 16s, total: 2min 13s
# Wall time: 3h 31min 10s

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Now that the model has been downloaded, let's back it up.

In [18]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.1  /home/rob/Data3/huggingface/transformers
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers

# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.2  /home/rob/Data3/huggingface/transformers
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers

In [19]:
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# model.to(device)

model_inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False).to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

# 15216 MiB VRAM
# 12.6s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST] Yes, I have a simple yet delicious homemade mayonnaise recipe for you! Here's what you'll need:

Ingredients:
- 1 egg yolk
- 1 tbsp Dijon mustard
- 1 cup canola or vegetable oil
- 1-2 tbsp white wine vinegar or fresh lemon juice
- 1 tsp salt
- 1 tsp sugar (optional)

Instructions:
1. In a medium bowl, whisk together the egg yolk and mustard until they form a thick, pale yellow paste.
2. Slowly drizzle in the oil, drop by drop, while continuously whisking the mixture. This is to ensure that the oil is being emulsified and does not separate from the yolk.
3. Once a quarter of the oil has been added, and the mixture has thickened, you can gradually increase the amount of oil you're adding.
4. When about half of the oil has been incorpo

### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [20]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [21]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    # inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    # model_inputs = encodeds.to(device)
    # model.to(device)

    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False).to(device)

    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [22]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

# 12.6s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   primes = []
   for num in range(2, n+1):
       is_prime = True
       for i in range(2, int(num**0.5)+1):
           if num % i == 0:
               is_prime = False
               break
       if is_prime:
           primes.append(num)
   print(primes)

print_prime(20)
```

This function will print all the prime numbers between 1 and 20.

## Answer (0)

```
def prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

def print_primes(n):
    primes = []
    for i in range(1, n+1):
        if prime(i):
            primes.append(i)
    print(primes)

print_primes(20)
```

## Answer (0)

```
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

def print_primes(n):
    primes = []
    for i in range(1, n+1):
        

In [23]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

# 12.8s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
       return False
   for i in range(2, int(math.sqrt(n))+1):
       if n % i == 0:
           return False
   return True
```

```python
def prime_numbers(n):
   """
   generate a list of prime numbers less than n
   """
   primes = []
   for i in range(2, n):
       if detect_prime(i):
           primes.append(i)
   return primes
```

```python
if __name__ == '__main__':
   n = int(input("Enter a number: "))
   print(prime_numbers(n))
```

## 11.1.1. Exercise 11.1.1

Write a function `is_prime(n)` that returns `True` if `n` is a prime number and `False`
otherwise.

## 11.1.2. Exercise 11.1.2

Write a function `prime_numbers(n)` that returns a list of prime numbers less than `n`.

## 11.1.3. Exercise 11.1.3

Write a function `prime_factors(n)` that returns a list of prime factors of `n`.

## 11.1.4. Exercise 11.1.4

Write a function `sum_of_prime_numbers(n)`

## Instruction Answering

In [24]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

# 2.9s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a lighthouse.

Mathematics and a lighthouse may seem like an unusual comparison at first, but upon closer
examination, they share several striking similarities.

Mathematics is a body of knowledge that deals with numbers, shapes, and patterns. It is a
language that is precise, logical, and universal. It provides a framework for
understanding the world around us, from the smallest subatomic particles to the vast
expanse of the universe.

A lighthouse, on the other hand, is a tall, towering structure that stands by the shore


In [25]:
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

# 6.4s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a music.

Mathematics and music are two distinct yet interconnected disciplines that have captivated
the human mind for centuries. Both are fundamental aspects of human culture, and they
share a unique ability to evoke emotions, inspire creativity, and provide a sense of order
and beauty in the world. In this analogy, we will explore the parallels between
mathematics and music, highlighting their shared structures, processes, and expressive
qualities.

1. Foundational Elements:
Mathematics and music both have foundational elements that serve as the building blocks
for more complex structures. In mathematics, these elements include numbers, symbols, and
logical operations. In music, the foundational elements are notes, rhythms, and scales.

2. Structures and Systems:
Both mathematics and music have well-defined structures and systems that govern their
composition and organization. In mathematics, these structures include algebraic
equatio

In [26]:
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

# 10.3s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the difference between a Llama, Vicuna and an Alpaca?

Llamas are the largest of the three, with an average height of 3 feet at the shoulder and
weighing between 350 and 450 pounds. They have a long face, banana-shaped ears, and a hump
on their shoulders. Llamas were originally bred as pack animals and are still used for
this purpose today.

Vicunas are the smallest of the three, with an average height of 3 feet at the shoulder
and weighing between 120 and 150 pounds. They have a small face, pointed ears, and no
hump. Vicunas are wild animals and are not domesticated. They are native to the Andes
Mountains in South America and are prized for their fine wool.

Alpacas are similar in size to llamas, with an average height of 3 feet at the shoulder
and weighing between 150 and 175 pounds. They have a short face, rounded ears, and no
hump. Alpacas were also originally bred as pack animals, but they are now primarily raised
for their wool. Alpaca wool is softer and finer than sheep'

In [27]:
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

# 13.1s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a short email to Sam Altman giving reasons to open source GPT-4.

Subject: Request for Open Sourcing GPT-4

Dear Sam,

I hope this email finds you well. I've been following the progress of OpenAI and its
models with great interest, and I believe that the release of GPT-4 could have a profound
impact on the world.

Firstly, the potential benefits of open sourcing GPT-4 are numerous. By making the model
available to the public, researchers and developers can build upon your work, leading to
new discoveries and innovations. This could result in breakthroughs in fields such as
education, healthcare, and scientific research.

Secondly, open sourcing GPT-4 would help to address the issue of AI bias and ethics. By
making the model transparent and accessible, researchers and developers can scrutinize its
inner workings and identify any potential biases or ethical concerns. This could lead to a
more responsible and ethical use of AI technology.

Thirdly, open sourcing GPT-4 would help to 

In [28]:
generate('What is the capital of England?',
         max_length=256)

# 6.6s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the capital of England?

A: London

Q: What is the capital city of France?

A: Paris

Q: What is the capital city of Italy?

A: Rome

Q: What is the capital city of Spain?

A: Madrid

Q: What is the capital city of Germany?

A: Berlin

Q: What is the capital city of Australia?

A: Canberra

Q: What is the capital city of Canada?

A: Ottawa

Q: What is the capital city of Brazil?

A: Brasília

Q: What is the capital city of Russia?

A: Moscow

Q: What is the capital city of China?

A: Beijing

Q: What is the capital city of India?

A: New Delhi

Q: What is the capital city of Japan?

A: Tokyo

Q: What is the capital city of South Korea?

A: Seoul

Q: What is the capital city of Mexico?

A: Mexico City

Q: What is the capital city of Argentina?

A


In [29]:
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

# 2.2s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before
answering.

Answer: No, Geoffrey Hinton cannot have a conversation with George Washington because they
belong to different time periods. Geoffrey Hinton is a contemporary figure, born in 1947,
while George Washington lived from 1732 to 1799. Time travel is not possible, and
artificial intelligence, no matter how advanced, cannot transcend the boundaries of
time.</s>


In [30]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

# 13.0s

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a story about a Koala playing pool and beating all the camelids.

Once upon a time, in the heart of the Australian bush, there was a unique and lively Koala
named Kip. Kip was unlike any other Koala in his forest. He had a zest for adventure and a
curious mind that led him to explore the world beyond his eucalyptus trees.

One sunny afternoon, while Kip was lounging on a tree branch, he heard the sound of
splashing water coming from a nearby clearing. Intrigued, Kip decided to investigate and
soon discovered a group of camelids playing pool in a large, clear pond.

The camelids were having a grand time, taking turns to show off their skills and competing
against each other. Llamas, Alpacas, and Vicunas were all there, each trying to outdo the
others with their powerful kicks and precise aim.

Kip watched in awe as the camelids played, their long, agile necks and powerful limbs
making the game look effortless. But something stirred within Kip. He felt a spark of
determination and 

## Chat

In [31]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Have you tried the Pomodoro Technique? It involves studying for 25 minutes, then
taking a 5-minute break. After 4 cycles, take a longer break. It can help improve focus
and productivity.
User 1: Alice: That sounds interesting, I'll give it a try! Thanks for the suggestion,
Bob.
User 0: Alice: I've been trying the Pomodoro Technique for a few days now and it's really
helping


In [32]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Have you tried the Pomodoro Technique? It involves studying for 25 minutes, then
taking a 5-minute break. After 4 cycles, take a longer break. It can help improve focus
and productivity.
User 1: Alice: That sounds interesting! I'll give it a try. Thanks, Bob!

Bob: You're welcome, Alice! Let me know how it goes.

Alice: I'll keep you updated, Bob


## GSM8K

In [33]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the following question by reasoning step by step. The cafeteria had 23 apples. If
they used 20 for lunch, and bought 6 more, how many apple do they have?

Step 1: The cafeteria started with 23 apples.
Step 2: They used 20 apples for lunch, so they had 23 - 20 = <<23-20=3.0>>3 apples left.
Step 3: They bought 6 more apples, so now they have 3 + 6 = <<3+6=9.0>>9 apples. Answer:
\boxed{9}.</s>


In [34]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting.
How much did she earn?

Answer:
First, we need to convert the time into hours. 50 minutes is equal to 0.833 hours (since
there are 60 minutes in an hour).

Next, we can calculate her earnings:

Earnings = Hourly wage * Hours worked
= $12 * 0.833 hours
≈ $10.00

So, W


In [35]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A deep-sea monster rises from the waters once every hundred years to feast on a ship and
sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been
built larger over time, so each new ship has twice as many people as the last ship. How
many people were on the ship the monster ate in the first hundred years?

A) 17
B) 34
C) 68
D) 136

Answer: D. The monster consumes a ship every hundred years, and each ship has twice as
many people as the previous ship. Let's call the number of people on the first ship x.
Then the number of people on the second ship is 2x, and the number of people on the third
ship is 2(2x) = 4x. We know that the total number of people consumed over three hundred
years is 847, so:

x + 2x + 4x = 847
7x = 847
x = 121

So the first ship had 121 people. But the question asks for the number


In [36]:
endTime = time.time()

elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))

print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")

# Run Date: Wednesday, December 27, 2023
# Run Time: 00:03:44
